# Main

# Train Fraud

## Conv VAE

### Fraud

In [2]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.conv_vae_model import ConvVae, vae_loss_function, print_num_params
from trainer.trainer_vae import VAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger
from utils.evaluation_utils import extract_recon_loss

# Build the config path
config_path = "configs/conv_vae/fraud_conv_vae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
conv_vae_config = config_parser["Conv_VAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = ConvVae(conv_vae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vae_loss_function

# Create trainer
trainer = VAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            train_batch = next(iter(dataloaders['train']))
            val_batch = next(iter(dataloaders['val']))
            train_recon_loss = extract_recon_loss(model, train_batch, trainer.device)
            val_recon_loss = extract_recon_loss(model, val_batch, trainer.device)
            wandb_logger.log_epoch(epoch, train_loss, val_loss, train_recon_loss, val_recon_loss)
        except Exception as e:
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

Initializing WandB run: conv-vae-fraud_20250313_173404 (Project: fraud-classification, Entity: alexkstern)


Loaded configuration from configs/conv_vae/fraud_conv_vae.config
Filtered dataset to class 1: 378 samples
Normalization statistics (calculated from class 1): {'Time': {'mean': 80790.48148148147, 'std': 48332.5139872635}, 'Amount': {'mean': 133.6764814814815, 'std': 276.3532237447719}}
Filtered dataset to class 1: 378 samples
Filtered dataset to class 1: 47 samples
Filtered dataset to class 1: 48 samples
Total number of trainable parameters: 62727
Models will be saved to: saved_models/conv_vae/fraud_conv_vae/20250313_173415


Epoch 1/150: Train Loss = 704.4488, Val Loss = 1022.8286
Model saved to saved_models/conv_vae/fraud_conv_vae/20250313_173415/best_model.pt
New best validation loss: 1022.8286


Epoch 2/150: Train Loss = 679.0434, Val Loss = 1022.5550
Model saved to saved_models/conv_vae/fraud_conv_vae/20250313_173415/best_model.pt
New best validation loss: 1022.5550


Epoch 3/150: Train Loss = 678.9731, Val Loss = 1022.5265
Model saved to saved_models/conv_vae/fraud_conv_vae/20250313_173415/best_model.pt
New best validation loss: 1022.5265


...

Epoch 148/150: Train Loss = 665.9193, Val Loss = 1006.9692


Epoch 149/150: Train Loss = 665.9197, Val Loss = 1006.9706


Epoch 150/150: Train Loss = 665.9197, Val Loss = 1006.9702
Model saved to saved_models/conv_vae/fraud_conv_vae/20250313_173415/final_model.pt
Training complete. Best validation loss: 1006.9674


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/recon_loss,▄▄▃▄▂█▂▁▃▃▄▄▃▃▃▃▃▇▆▁▁▂▄▅▄▄▂▁▄▄▄▄▃▅▃▅▂▆▆▁
train/total_loss,█████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/recon_loss,████████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/total_loss,█████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,150
train/recon_loss,4001.76562
train/total_loss,665.91969
val/recon_loss,10876.7793
val/total_loss,1006.97016


Training session complete.


### Normal

In [1]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.conv_vae_model import ConvVae, vae_loss_function, print_num_params
from trainer.trainer_vae import VAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger
from utils.evaluation_utils import extract_recon_loss

# Build the config path
config_path = "configs/conv_vae/normal_conv_vae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
conv_vae_config = config_parser["Conv_VAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = ConvVae(conv_vae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vae_loss_function

# Create trainer
trainer = VAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            train_batch = next(iter(dataloaders['train']))
            val_batch = next(iter(dataloaders['val']))
            train_recon_loss = extract_recon_loss(model, train_batch, trainer.device)
            val_recon_loss = extract_recon_loss(model, val_batch, trainer.device)
            wandb_logger.log_epoch(epoch, train_loss, val_loss, train_recon_loss, val_recon_loss)
        except Exception as e:
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

/home/alexkstern/miniconda3/envs/credit_vae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Initializing WandB run: conv-vae-normal_20250313_175704 (Project: fraud-classification, Entity: alexkstern)


wandb: Currently logged in as: alexkstern (alexksternteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loaded configuration from configs/conv_vae/normal_conv_vae.config
Filtered dataset to class 0: 12000 samples
Normalization statistics (calculated from class 0): {'Time': {'mean': 94364.65358333333, 'std': 47365.815157589255}, 'Amount': {'mean': 87.60478666666666, 'std': 240.59403081682598}}
Filtered dataset to class 0: 12000 samples
Filtered dataset to class 0: 1500 samples
Filtered dataset to class 0: 1500 samples
Total number of trainable parameters: 62727
Models will be saved to: saved_models/conv_vae/normal_conv_vae/20250313_175713


Epoch 1/30: Train Loss = 31.6793, Val Loss = 32.5413
Model saved to saved_models/conv_vae/normal_conv_vae/20250313_175713/best_model.pt
New best validation loss: 32.5413


...

Epoch 28/30: Train Loss = 31.5505, Val Loss = 32.5413


Epoch 29/30: Train Loss = 31.5505, Val Loss = 32.5413


Epoch 30/30: Train Loss = 31.5505, Val Loss = 32.5413
Model saved to saved_models/conv_vae/normal_conv_vae/20250313_175713/final_model.pt
Training complete. Best validation loss: 32.5407


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/recon_loss,▃█▂▂▂▂▂▃▃▃▃▂▄▁▂▁▂▂▂▁▁▁▂▁▂▃▂▂▂▂
train/total_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/recon_loss,█▁▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val/total_loss,█▁▇███████████████████████████
epoch,30
train/recon_loss,170.05019
train/total_loss,31.55054
val/recon_loss,199.38646
val/total_loss,32.54126


Training session complete.


## Transformer VAE

### Fraud

In [1]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.transformer_vae_model import TransformerVae, vae_loss_function, print_num_params
from trainer.trainer_vae import VAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger
from utils.evaluation_utils import extract_recon_loss

# Build the config path
config_path = "configs/transformer_vae/fraud_transformer_vae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
transformer_vae_config = config_parser["Transformer_VAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = TransformerVae(transformer_vae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vae_loss_function

# Create trainer
trainer = VAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            train_batch = next(iter(dataloaders['train']))
            val_batch = next(iter(dataloaders['val']))
            train_recon_loss = extract_recon_loss(model, train_batch, trainer.device)
            val_recon_loss = extract_recon_loss(model, val_batch, trainer.device)
            wandb_logger.log_epoch(epoch, train_loss, val_loss, train_recon_loss, val_recon_loss)
        except Exception as e:
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

/home/alexkstern/miniconda3/envs/credit_vae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Initializing WandB run: transformer-vae-fraud_20250313_190718 (Project: fraud-classification, Entity: alexkstern)


wandb: Currently logged in as: alexkstern (alexksternteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loaded configuration from configs/transformer_vae/fraud_transformer_vae.config
Filtered dataset to class 1: 378 samples
Normalization statistics (calculated from class 1): {'Time': {'mean': 80790.48148148147, 'std': 48332.5139872635}, 'Amount': {'mean': 133.6764814814815, 'std': 276.3532237447719}}
Filtered dataset to class 1: 378 samples
Filtered dataset to class 1: 47 samples
Filtered dataset to class 1: 48 samples
Total number of trainable parameters: 147879
Models will be saved to: saved_models/transformer_vae/fraud_transformer_vae/20250313_190737


Epoch 1/100: Train Loss = 682.0477, Val Loss = 1022.8600
Model saved to saved_models/transformer_vae/fraud_transformer_vae/20250313_190737/best_model.pt
New best validation loss: 1022.8600


...

Epoch 98/100: Train Loss = 678.9608, Val Loss = 1022.5318


Epoch 99/100: Train Loss = 678.9608, Val Loss = 1022.5318
Model saved to saved_models/transformer_vae/fraud_transformer_vae/20250313_190737/best_model.pt
New best validation loss: 1022.5318


Epoch 100/100: Train Loss = 678.9608, Val Loss = 1022.5318
Model saved to saved_models/transformer_vae/fraud_transformer_vae/20250313_190737/final_model.pt
Training complete. Best validation loss: 1022.5318


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/recon_loss,▂▃▃▂▃▃▄▅▃▁▆▆▄▃▇█▂▄▂▂▅▁▅▁▅▃▃▄▃▄▃▄▅▄▄▄▃▂▃▅
train/total_loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/recon_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/total_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,100
train/recon_loss,9307.76562
train/total_loss,678.96082
val/recon_loss,11025.65723
val/total_loss,1022.5318


Training session complete.


### Normal

In [ ]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.transformer_vae_model import TransformerVae, vae_loss_function, print_num_params
from trainer.trainer_vae import VAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger
from utils.evaluation_utils import extract_recon_loss

# Build the config path
config_path = "configs/transformer_vae/normal_transformer_vae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
transformer_vae_config = config_parser["Transformer_VAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = TransformerVae(transformer_vae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vae_loss_function

# Create trainer
trainer = VAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            train_batch = next(iter(dataloaders['train']))
            val_batch = next(iter(dataloaders['val']))
            train_recon_loss = extract_recon_loss(model, train_batch, trainer.device)
            val_recon_loss = extract_recon_loss(model, val_batch, trainer.device)
            wandb_logger.log_epoch(epoch, train_loss, val_loss, train_recon_loss, val_recon_loss)
        except Exception as e:
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

## Conv VQ VAE

### Fraud

In [ ]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.conv_vqvae_model import ConvVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger
from utils.evaluation_utils import extract_recon_loss

# Build the config path
config_path = "configs/conv_vqvae/fraud_conv_vqvae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
conv_vqvae_config = config_parser["Conv_VQVAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = ConvVQVAE(conv_vqvae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create trainer
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log metrics to WandB
        try:
            # For VQVAE, we only log the total losses since we don't have separate recon_loss extraction
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        except Exception as e:
            print(f"Error logging to WandB: {e}")
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

### Normal


In [ ]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.conv_vqvae_model import ConvVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger

# Build the config path
config_path = "configs/conv_vqvae/normal_conv_vqvae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
conv_vqvae_config = config_parser["Conv_VQVAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = ConvVQVAE(conv_vqvae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create trainer
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log metrics to WandB
        try:
            # For VQVAE, we only log the total losses since we don't have separate recon_loss extraction
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        except Exception as e:
            print(f"Error logging to WandB: {e}")
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

## Transformer VQVAE

### Fraud

In [ ]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.transformer_vqvae_model import TransformerVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger

# Build the config path
config_path = "configs/transformer_vqvae/fraud_transformer_vqvae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
transformer_vqvae_config = config_parser["Transformer_VQVAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = TransformerVQVAE(transformer_vqvae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create trainer
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            # For VQVAE, we only log the total losses 
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        except Exception as e:
            print(f"Error logging to WandB: {e}")
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

### Normal

In [ ]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.transformer_vqvae_model import TransformerVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger

# Build the config path
config_path = "configs/transformer_vqvae/normal_transformer_vqvae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
transformer_vqvae_config = config_parser["Transformer_VQVAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = TransformerVQVAE(transformer_vqvae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create trainer
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            # For VQVAE, we only log the total losses 
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        except Exception as e:
            print(f"Error logging to WandB: {e}")
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")